# В конце этого семинара - домашка!

# PyTorch

## __Tensors__

Одно из основных понятий в PyTorch -- это __Tenosor__. 

https://pytorch.org/docs/master/tensors.html

__Tensor__ -- это такой же массив, как и в __numpy.array__, размерность и тип данных которого мы можем задать. Tensor в отличие от numpy.array может вычисляться на __GPU__.

In [9]:
import numpy as np
import torch

In [10]:
N = 100
D_in = 50

dtype = torch.float
device = torch.device("cpu")
# device = torch.device("cuda:0") # Uncomment this to run on GPU

x = np.random.randn(N, D_in)
x_torch = torch.randn(N, D_in, device=device, dtype=dtype)

In [ ]:
x

array([[ 0.22698361, -0.92910466, -0.3409152 , ...,  0.30752526,
         0.66814506, -0.15665968],
       [ 0.24220215,  0.69735813,  0.14206001, ..., -0.27255869,
         0.07161657,  0.8169845 ],
       [-0.28821643, -0.63891994,  1.32400691, ..., -0.38076706,
         0.31225951,  0.38870691],
       ...,
       [ 1.38097763,  1.0240416 , -0.61224253, ...,  1.8633075 ,
         1.08992317,  0.73212091],
       [ 0.57582689,  1.03744011,  0.77708304, ..., -1.46306664,
        -0.551786  ,  0.23369634],
       [ 0.53723579,  0.67381284, -0.34782831, ..., -0.40658037,
        -0.95111284, -1.09284501]])

In [ ]:
x_torch

tensor([[ 0.4162,  1.8735,  0.3687,  ...,  0.2289,  1.0640,  0.3105],
        [ 0.4008, -0.9234, -0.4625,  ..., -1.7048,  0.5290, -0.3508],
        [-0.0953, -0.1071,  0.2812,  ..., -1.1540,  1.1987,  0.2639],
        ...,
        [-0.4731,  1.2565,  0.5293,  ...,  0.3280, -0.4088,  0.3165],
        [ 0.9807,  1.0674, -0.8862,  ..., -0.7327, -1.6180,  0.9946],
        [ 0.6781, -0.8115,  1.8938,  ..., -0.7975,  0.3126, -0.1241]])

In [ ]:
x_torch = torch.Tensor(np.ones((N, D_in)))
x_torch

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [ ]:
x_torch = torch.FloatTensor([1, 2, 3])
x_torch

tensor([1., 2., 3.])

In [ ]:
x1 = torch.IntTensor([1, 2, 3])
x2 = torch.FloatTensor([3, 4, 5])

In [ ]:
print(x1)
print(x2)

tensor([1, 2, 3], dtype=torch.int32)
tensor([3., 4., 5.])


В PyTorch можно найти много операций, которые похожи на то, что есть в numpy :
```
- torch.add (np.add) -> сложение тензоров (поэлементное)
- torch.sub (np.subtract) -> вычитание (поэлементное)
- torch.mul (np.multiply) -> умнажение скаляров / матриц (поэлементное)
- torch.mm (np.matmul) -> перемножение матриц
- torch.ones (np.ones) -> создание тензора из единиц
```

In [ ]:
# Давайте попробуем вышепересчисленные операции

In [ ]:
x1 = torch.FloatTensor([[1, 2, 3], [4, 5, 6]])

In [ ]:
x2 = torch.FloatTensor([[7, 8], [9, 1], [2, 3]])

In [ ]:
out = torch.mm(x1, x2)
out

tensor([[31., 19.],
        [85., 55.]])

```
- torch.view (np.reshape) -> изменения порядка элементов в тензоре, не путать с транспонированием.
```

## Dynamic Computational Graph

После того, как были реализованы архитектура модели и весь процес обучения и валидация сети, при запуске кода в PyTorch происходят следующие этапы:

1. Строится вычислительный граф (направленный ациклический граф), где каждое ребро, ведущее к дргуому узлу, -- это тензор, а узел - это выполнение операции над данным тензором.

<img src="./images/Graph.png" alt="Drawing" style="width: 300px;"/>

Реализуем двухслойную сеть для задачи регрессии. И граф для такой архитектуры бдует выглядить следующим образом:

<img src="./images/RegGraph.png" alt="Drawing" />

In [ ]:
batch_size = 64
input_size = 3
hidden_size = 2
output_size = 1

In [ ]:
# Create random input and output data
x = torch.randn(batch_size, input_size, device=device, dtype=dtype)
y = torch.randn(batch_size, output_size, device=device, dtype=dtype)

# Randomly initialize weights
w1 = torch.randn(input_size, hidden_size, device=device, dtype=dtype)
w2 = torch.randn(hidden_size, output_size, device=device, dtype=dtype)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    #TODO
    h_1 = x.mm(w1)
    h_relu = h_1.clamp(min=0)
    out = h_relu.mm(w2)
    
    # Compute and print loss
    loss = (out - y).pow(2).sum().item()
    
    # Backward pass: 
    dloss_dout = 2 * (out - y)
    
    grad_w2 = h_relu.t().mm(dloss_dout) 
    
    grad_h_relu = dloss_dout.mm(w2.t())
    
    grad_h_relu[h_1 < 0] = 0
    
    grad_w1 = x.t().mm(grad_h_relu)
    
    
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    if t % 100 == 99:
        print('Loss on iteration {} = {}'.format(t, loss))
    

Loss on iteration 99 = 89.06915283203125
Loss on iteration 199 = 86.82075500488281
Loss on iteration 299 = 84.73787689208984
Loss on iteration 399 = 82.80679321289062
Loss on iteration 499 = 81.01509094238281


In [ ]:
loss

81.01509094238281

## Autograd

2. Еще одно фундаментальное понятие и важный элемент при построении графа -- это __Autograd__ -- автоматическое дифференцирование.

Для того чтобы с помощью стохастического градиентного спуска обновить обучаемые параметры сети, нужно посчитать градиенты. И как известно, обновление весов, которые учавтсвуют в нескольких операциях, происходит по `правилу дифференцирования сложной функции` (цепное правило или __chain rule__).

<img src="./images/RegChainRule.png" alt="Drawing" />

То есть (1) вычислительный граф позволяет определить последовательность операций, а (2) автоматическое дифференцирование посчитать нужные градиенты.

Если бы `Autograd` не было, то тогда backprop надо было бы реализовывать самим, и как это бы выглядело?

Рассмотрим на примере, как посчиать градиенты для весов из входного слоя, где входной вектора `X` состоит из 3-х компонент. А входной слой вторую размерность имеет равной 2. 

После чего это идет в `ReLU`, но для простоты опустим на время ее, и посмотрим как дальше это идет по сети.

Ниже написано, как это все вычисляется и приводит нас к значению целевой функции для одного наблюдения

<img src="./images/1.png" alt="1" style="width: 600px;"/>

Тогда, чтобы посчитать градиент по первому элементу из обучаемой матрицы на первом слое, необходимо взять производоную у сложной функции. А этот как раз делается по `chain rule`: сначала берем у внешней, потом спускаемся на уровень ниже, и так пока не додйдем до то функции, после которой эта перменная уже нигде не участвует:

<img src="./images/2.png" alt="2" style="width: 400px;"/>

Перепишем это все в матричном виде, то есть сделаем аналог вида матрицы весов из первого слоя, но там уже будут её градиенты, котоыре будут нужны чтобы как раз обновить эти веса:

<img src="./images/3.jpg" alt="3" style="width: 600px;"/>

Как видно, здесь можно вектор X вынести, то есть разделить на две матрицы:

<img src="./images/4.jpg" alt="4" style="width: 500px;"/>

То есть уже видно, что будем траспонировать входной вектор(матрицу). Но надо понимать, что в реальности у нас не одно наблюдение в батче, а несколько, тогда запись немного изменит свой вид:

<img src="./images/5.jpg" alt="5" style="width: 500px;"/>

Теперь мы видим, как на самом деле вычисляется вот те самые частные производные для вектора X, то есть видно, как математически это можно записать, а именно:

<img src="./images/6.jpg" alt="6" style="width: 500px;"/>

<img src="./images/7.jpg" alt="7" style="width: 500px;"/>

Уже можно реализовать. Понятно, что транспонируется, что нет, и что на что умножается.

Но помним про ReLU. Для простоты опустили, но теперь её учесть будет легче. 

Так как после первого слоя идет ReLU, а значит, занулились те выходы первого слоя, которые были __меньше__ нуля. Получается, что во второй слой не все дошло, тогда нужно обнулить, что занулил ReLU. 

Что занулил ReLU, мы можем выяснить при `forward pass`, а где именно поставить нули, то надо уже смотреть относительно `backward propagation`, на том выходе, где последний раз участвовал выход после ReLU, то есть:

<img src="./images/8.jpg" alt="8" style="width: 600px;"/>

Благодаря `Autograd` реализацию `chain rule` можно избежать, так как для более сложных нейронных сетей вручную такое реализовать сложно, при этом сделать это эффективным.

Для того чтобы PyTorch понял, за какими переменными надо "следить", то есть указать, что именно "эти" переменные являются обучаемыми, необходимо при создании тензора в качестве аттрибута указать __requires_grad=True__:

In [ ]:
w1 = torch.randn(input_size, hidden_size, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(hidden_size, output_size, device=device, dtype=dtype, requires_grad=True)

In [ ]:
learning_rate = 1e-6
for t in range(500):
    y_pred = x.mm(w1).clamp(min=0).mm(w2)

    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
    
    # Теперь подсчет градиентов для весов происходит при вызове backward
    loss.backward()
   
    # Обновляем значение весов, но укзаываем, чтобы PyTorch не считал эту операцию, 
    # которая бы учавствовала бы при подсчете градиентов в chain rule
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad
        
        # Теперь обнуляем значение градиентов, чтобы на следующем шаге 
        # они не учитывались при подсчете новых градиентов,
        # иначе произойдет суммирвоание старых и новых градиентов
        w1.grad.zero_()
        w2.grad.zero_()

99 229.716796875
199 214.16131591796875
299 200.17263793945312
399 187.5623779296875
499 176.169189453125


Осталось еще не вручную обновлять веса, а использовать адаптивные методы градинетного спсука. Для этого нужно использовать модуль __optim__. А помимо оптимайзера, еще можно использовать готовые целевые функции из модлуя __nn__.

In [ ]:
w1 = torch.randn(input_size, hidden_size, device=device, dtype=dtype, requires_grad=True)
w2 = torch.randn(hidden_size, output_size, device=device, dtype=dtype, requires_grad=True)

In [ ]:
import torch.optim as optim

loss_fn = torch.nn.MSELoss(reduction='sum')

learning_rate = 1e-6
optimizer = torch.optim.Adam([w1, w2], lr=learning_rate)

for t in range(500):
    optimizer.zero_grad()
    
    y_pred = x.mm(w1).clamp(min=0).mm(w2)
    
    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())
    
    loss.backward()
   
    optimizer.step()

99 241.64454650878906
199 241.53431701660156
299 241.424072265625
399 241.31387329101562
499 241.2037353515625


После того, как мы сделали backward, в этот момент посчитались градиенты и граф уничтожился, то есть стёрлись все пути, которые связывали тензоры между собой. Это значит, что еще раз backward сделать не поулчится, будет ошибка. Но если вдруг нужно считать градиенты еще раз, то нужно при вызове backward задать `retain_graph=True`.

Еще важный аттрибут, который есть у Tensor -- это `grad_fn`. В этом аттрибуте указывается та функция, посредством которой был создан этот тензор. Так PyTorch понимает, как именно считать по нему градиент.

In [ ]:
y_pred.grad_fn

Также можно контролировать, должны ли градиенты течь или нет.

In [ ]:
x = torch.tensor([1.], requires_grad=True)
with torch.no_grad():
    with torch.enable_grad():
        y = x * 2
y.requires_grad

True

## Почему Backprop надо понимать

1. Backprop позволяет понимать, как те или иные операции, сложные конструкции в сети влияют на обнолвение весов.
Почему лучше сделать конкатенацию тензоров, а не поэлементное сложение. Для этого нужно посмотреть на backprop, как будут обновляться веса.

2. Даже на таком маленьком пример двуслойной MLP можно уже увидеть, когда `ReLU`, как функция активация, не очень хорошо применять. Если разреженные данные, то получить на выходе много нулей вероятнее, чем при использовании `LeakyReLU`, то есть градиенты будут нулевыми и веса никак не будут обновляться => сеть не обучается!

3. В архитектуре могут встречаться недифференцируемые операции, и первое - это нужно понять, потому что при обучении сети это может быть не сразу заметно, просто качество модели будет плохое, и точность хорошую не поулчится достичь.

Например, в одной из статей было предложено в качестве механизма внимания применить распределение Бернулли, которое умножается на выход промежуточного слоя сети. И эта опреация недифференцируема, нужно реализовывать backprop самим, тем самым обеспечить корректное протекание градиентов.


<img src="./images/Bernoulli.png" alt="8" style="width: 600px;"/>

Так же любая статья, которая предлагет новую целевую функцию для той или иной задачи, там всегда будут представлены градиенты, чтобы было понимание, как это влияет на обновление весов. Не просто так !

<img src="./images/BernoulliBackProp.png" alt="8" style="width: 600px;"/>

## nn.Module

В предыдущем примере архитектуру сети создавали используя последовательной способ объявления слоев сети -- `nn.Sequential`.

Но еще можно это сделать более гибким подходом:

In [ ]:
import torch.nn as nn

In [ ]:
class TwoLayerNet(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        """
        TwoLayerNet наследуется от nn.Module и тем самым полчаем возможность
        переопределять методы класса.
        В конструктуре создаем слои (обучаемые веса) и другие нужные перменные/функции,
        которые нужны для модели
        """
        super(TwoLayerNet, self).__init__()
        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, x):
        """
        Метод forward отвечает за прямое распростронение модели, 
        поэтому данный метод нужно переопределять обязательно, 
        чтобы задать логику прямого распростронения. 
        Именно в этот момент начинает строится динамический граф
        """
        h_relu = self.linear1(x).clamp(min=0)
        y_pred = self.linear2(h_relu)
        
        return y_pred

In [ ]:
batch_size = 64
input_size = 1000
hidden_size = 100
output_size = 10

x = torch.randn(batch_size, input_size, device=device, dtype=dtype)
y = torch.randn(batch_size, output_size, device=device, dtype=dtype)

model = TwoLayerNet(input_size, hidden_size, output_size)

loss_fn = torch.nn.MSELoss(reduction='sum')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)

for t in range(500):
    y_pred = model(x)

    loss = loss_fn(y_pred, y)
    if t % 100 == 99:
        print(t, loss.item())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

99 1.267189621925354
199 0.013050442561507225
299 0.0003220129001419991
399 1.1580568752833642e-05
499 5.351666345632111e-07


# Домашние задание

1. Добавить Bias и посчитать для них градиенты.
2. Сравнить градинеты с тем, как считает PyTorch AutoGrad.

In [11]:
batch_size = 64
input_size = 3
hidden_size = 2
output_size = 1

In [12]:
w1_src = torch.randn(input_size, hidden_size, device=device, dtype=dtype)
w2_src = torch.randn(hidden_size, output_size, device=device, dtype=dtype)
b1_src = torch.randn(1, hidden_size, device=device, dtype=dtype)
b2_src = torch.randn(1, output_size, device=device, dtype=dtype)

In [13]:
# Create random input and output data
x = torch.randn(batch_size, input_size, device=device, dtype=dtype)
y = torch.randn(batch_size, output_size, device=device, dtype=dtype)

# Randomly initialize weights
w1 = w1_src.clone()
w2 = w2_src.clone()
b1 = b1_src.clone()
b2 = b2_src.clone()

#print(b1.shape)

learning_rate = 1e-6
for t in range(500):
    # Forward pass: compute predicted y
    #TODO
    h_1 = x.mm(w1) + b1
    #print(h_1.shape)
    h_relu = h_1.clamp(min=0)
    out = h_relu.mm(w2) + b2
    #print(out.shape)
    
    # Compute and print loss
    loss = (out - y).pow(2).sum().item()
    
    # Backward pass: 
    dloss_dout = 2 * (out - y)
    
    grad_w2 = h_relu.t().mm(dloss_dout) 

    grad_b2 = dloss_dout.sum(dim=0).reshape(1, 1)
    #print(grad_b2.shape)
    
    grad_h_relu = dloss_dout.mm(w2.t())
    
    grad_h_relu[h_1 < 0] = 0
    
    grad_w1 = x.t().mm(grad_h_relu)

    grad_b1 = grad_h_relu.sum(dim=0).reshape(1, 2)
    #print(b1.shape)
    #print(grad_b1.shape)

    #print(b2.shape)
    #print(grad_b2.shape)
    
    
    w1 -= learning_rate * grad_w1
    w2 -= learning_rate * grad_w2
    b1 -= learning_rate * grad_b1
    b2 -= learning_rate * grad_b2
    if t % 100 == 99:
        print('Loss on iteration {} = {}'.format(t, loss))

Loss on iteration 99 = 161.16864013671875
Loss on iteration 199 = 155.29055786132812
Loss on iteration 299 = 149.81507873535156
Loss on iteration 399 = 144.70809936523438
Loss on iteration 499 = 139.9320068359375


In [14]:
loss

139.9320068359375

### PyTorch AutoGrad

In [16]:
w1 = w1_src.clone()
w1.requires_grad = True
w2 = w2_src.clone()
w2.requires_grad = True
b1 = b1_src.clone()
b1.requires_grad = True
b2 = b2_src.clone()
b2.requires_grad = True

In [17]:
learning_rate = 1e-6
for t in range(500):
    y_pred = (x.mm(w1) + b1).clamp(min=0).mm(w2) + b2

    loss = (y_pred - y).pow(2).sum()
    if t % 100 == 99:
        print(t, loss.item())
    
    # Теперь подсчет градиентов для весов происходит при вызове backward
    loss.backward()
   
    # Обновляем значение весов, но укзаываем, чтобы PyTorch не считал эту операцию, 
    # которая бы учавствовала бы при подсчете градиентов в chain rule
    with torch.no_grad():
        w1 -= learning_rate * w1.grad
        w2 -= learning_rate * w2.grad

        b1 -= learning_rate * b1.grad
        b2 -= learning_rate * b2.grad
        
        # Теперь обнуляем значение градиентов, чтобы на следующем шаге 
        # они не учитывались при подсчете новых градиентов,
        # иначе произойдет суммирвоание старых и новых градиентов
        w1.grad.zero_()
        w2.grad.zero_()
        b1.grad.zero_()
        b2.grad.zero_()

99 161.16864013671875
199 155.29055786132812
299 149.81507873535156
399 144.70809936523438
499 139.9320068359375


In [ ]:
# Как видим, сходимость вроде есть.